In [ ]:
# !pip install accelerate
# !pip install peft==0.4.0
# !pip install transformers==4.31.0
# !pip install trl==0.4.7
# !pip install bitsandbytes==0.40.2
# !pip install evaluate rouge_score

## Importing Libraries

In [ ]:
import os
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from evaluate import load

## Loading Dataset

In [ ]:
train_dataset = pd.read_csv("samsum-train.csv")
val_dataset = pd.read_csv("samsum-validation.csv")
test_dataset = pd.read_csv("samsum-test.csv")

## Formatting the Dataset

In [ ]:
def format_samsum_prompt(dialogue, summary=None):
    prompt = f"Dialogue: {dialogue}\ntl;dr:"
    if summary:
        prompt += f" {summary}"
    return prompt

train_texts = []
val_texts = []
test_texts = []

for _, row in train_dataset.iterrows():
    train_texts.append(format_samsum_prompt(row['dialogue'], row['summary']))
train_dataset['text'] = train_texts

for _, row in val_dataset.iterrows():
    val_texts.append(format_samsum_prompt(row['dialogue'], row['summary']))
val_dataset['text'] = val_texts

for _, row in test_dataset.iterrows():
    test_texts.append(format_samsum_prompt(row['dialogue'], row['summary']))
test_dataset['text'] = test_texts

train_dataset = Dataset.from_pandas(train_dataset)
val_dataset = Dataset.from_pandas(val_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

## Loading the Model

In [ ]:
model_name = "gpt2"
new_model = "gpt2-summarization-finetuned"

## LoRa Parameters

In [ ]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

## SFT Parameters

In [ ]:
max_seq_length = 1024
packing = False
device_map = "auto"

## Training Arguments


In [ ]:
output_dir = "./results"
num_train_epochs = 2
fp16 = True
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
weight_decay = 0.001
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

## Loading Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## Setting Arguments

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn", "c_proj"]
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=200,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_total_limit=2,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:299: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,4.045200,3.469487
200,3.110400,3.036114
300,2.973100,2.928584
400,2.919200,2.860091
500,2.908800,2.827345
600,2.759700,2.783965
700,2.716900,2.764311
800,2.787400,2.756032
900,2.697100,2.737889
1000,2.720400,2.722039


TrainOutput(global_step=7366, training_loss=2.7182472455834943, metrics={'train_runtime': 2292.9134, 'train_samples_per_second': 12.85, 'train_steps_per_second': 3.213, 'total_flos': 3138261605019648.0, 'train_loss': 2.7182472455834943, 'epoch': 2.0})

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    max_length=2048,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    num_return_sequences=1,
)

In [ ]:
def generate_summary(dialogue):
    prompt = format_samsum_prompt(dialogue)
    result = pipe(prompt)
    return result[0]['generated_text'].split("tl;dr:")[1].strip()

In [ ]:
def get_summary_by_index(index):
    if index < 0 or index >= len(test_dataset):
        print(f"Invalid index. Please enter a number between 0 and {len(test_dataset)-1}")
        return

    test_dialogue = test_dataset[index]['dialogue']
    actual_summary = test_dataset[index]['summary']
    generated_summary = generate_summary(test_dialogue)

    print(f"\nExample at index {index}:")
    print("-" * 50)
    print("Dialogue:")
    print(test_dialogue)
    print("\nGenerated Summary:")
    print(generated_summary)
    print("\nActual Summary:")
    print(actual_summary)
    print("=" * 80)

In [ ]:
get_summary_by_index(604)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Example at index 604:
--------------------------------------------------
Dialogue:
Milena: How was the presentation?
Kate: very good
Regina: yes, we had a good feedback
Milena: how many people came to listen to you?
Regina: 3 but we new it will be very cosy, friendly thing
Milena: sometimes a small but interested audience is better than random bored people
Kate: exactly

Generated Summary:
Kate and Regina had a good presentation. Milena and Kate had a good feedback. Milena is not interested in people who are bored with music. Milena will introduce everyone to 3 people. Milena will also have a good audience. Milena will introduce everyone to music.

Actual Summary:
Kate, Milena and Regina's presentation went well. Three people came to listen to them.


In [ ]:
# adapter_path = os.path.join(new_model, "adapter")
# os.makedirs(adapter_path, exist_ok=True)

# trainer.model.save_pretrained(
#     adapter_path,
#     safe_serialization=True
# )

# tokenizer.save_pretrained(new_model)

# model.config.save_pretrained(new_model)

# print(f"Model adapter saved to: {adapter_path}")
# print(f"Tokenizer and config saved to: {new_model}")

# repo_name = "saadsohail/gpt2-text-summarization"
# try:
#     trainer.model.push_to_hub(
#         repo_name,
#         use_auth_token=True,
#         safe_serialization=True
#     )

#     tokenizer.push_to_hub(
#         repo_name,
#         use_auth_token=True
#     )
#     print(f"Successfully pushed to hub: {repo_name}")
# except Exception as e:
#     print(f"Hub push failed: {e}")
#     print("Model saved locally only")

In [ ]:
pipe = pipeline(task="text-generation",
               model=model,
               tokenizer=tokenizer,
               max_length=200,
               temperature=0.7,
               top_p=0.9,
               repetition_penalty=1.2)

In [ ]:
rouge = load('rouge')

MAX_ROUGE_SAMPLES = 50

predictions = []
references = []


for item in list(test_dataset)[:MAX_ROUGE_SAMPLES]:
    dialogue = item['dialogue']
    prompt = format_samsum_prompt(dialogue)
    result = pipe(prompt)
    try:
        generated_summary = result[0]['generated_text'].split("tl;dr:")[1].strip()
    except:
        generated_summary = result[0]['generated_text'].split("Dialogue:")[1].strip()
    predictions.append(generated_summary)
    references.append(item['summary'])

print(f"\nCalculating ROUGE scores on {len(predictions)} samples...")

rouge_scores = rouge.compute(
    predictions=predictions,
    references=references,
    use_stemmer=True
)

print("\nROUGE Scores:")
for metric, value in rouge_scores.items():
    print(f"{metric}: {value:.4f}")



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 200, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 369, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 436, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 360, but


Calculating ROUGE scores on 50 samples...

ROUGE Scores:
rouge1: 0.1048
rouge2: 0.0092
rougeL: 0.0742
rougeLsum: 0.0741
